# Imports

In [4]:
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import pandas as pd
import duckdb

In [5]:
#print(sys.path)

____

___

# Read From duckbd

In [7]:
con = duckdb.connect("data/test_testdb_5_min.duckdb", read_only=True)
#print(con.execute("SELECT * FROM candles ORDER BY timestamp DESC LIMIT 5").fetchall())
query = """SELECT * 
           FROM candles
        """
query_results = con.execute(query).fetchall()

In [8]:
print(query_results)

[(datetime.datetime(2025, 5, 18, 13, 45), 103901.6, 103901.6, 103838.7, 103844.0, 18.99174218), (datetime.datetime(2025, 5, 18, 13, 50), 103832.7, 103834.8, 103818.1, 103821.0, 0.3781236), (datetime.datetime(2025, 5, 18, 13, 55), 103821.0, 103821.1, 103818.0, 103821.1, 3.03764702), (datetime.datetime(2025, 5, 18, 14, 0), 103821.1, 103821.1, 103809.0, 103811.0, 0.26115685), (datetime.datetime(2025, 5, 18, 14, 5), 103811.0, 103811.0, 103800.6, 103800.6, 0.01958802), (datetime.datetime(2025, 5, 18, 14, 10), 103800.6, 103800.6, 103778.5, 103778.5, 0.06754324), (datetime.datetime(2025, 5, 18, 14, 15), 103778.6, 103778.6, 103778.5, 103778.6, 0.01457672), (datetime.datetime(2025, 5, 18, 14, 20), 103778.6, 103778.6, 103778.5, 103778.6, 0.03316182), (datetime.datetime(2025, 5, 18, 14, 25), 103778.6, 103810.9, 103778.5, 103810.9, 0.21634397), (datetime.datetime(2025, 5, 18, 14, 30), 103810.9, 103810.9, 103810.9, 103810.9, 0.00017988), (datetime.datetime(2025, 5, 18, 14, 35), 103884.0, 103884.2, 

In [9]:
con.close()

___

# Pipe the results to a Pandas Dataframe

In [10]:
df = pd.DataFrame(query_results)

In [11]:
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']

In [12]:
df.tail(30)

,Datetime,Open,High,Low,Close,Volume
0,2025-05-18 13:45:00,103901.6,103901.6,103838.7,103844.0,18.991742
1,2025-05-18 13:50:00,103832.7,103834.8,103818.1,103821.0,0.378124
2,2025-05-18 13:55:00,103821.0,103821.1,103818.0,103821.1,3.037647
3,2025-05-18 14:00:00,103821.1,103821.1,103809.0,103811.0,0.261157
4,2025-05-18 14:05:00,103811.0,103811.0,103800.6,103800.6,0.019588
5,2025-05-18 14:10:00,103800.6,103800.6,103778.5,103778.5,0.067543
6,2025-05-18 14:15:00,103778.6,103778.6,103778.5,103778.6,0.014577
7,2025-05-18 14:20:00,103778.6,103778.6,103778.5,103778.6,0.033162
8,2025-05-18 14:25:00,103778.6,103810.9,103778.5,103810.9,0.216344
9,2025-05-18 14:30:00,103810.9,103810.9,103810.9,103810.9,0.000180


In [13]:
len(df)

11

____


____

___

In [ ]:
# strategy/ta.py
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import duckdb
import pandas as pd
import pandas_ta as ta
from datetime import datetime, timedelta

DB_PATH = "data/market_data.duckdb"

def load_recent_candles(hours=200):
    con = duckdb.connect(DB_PATH)
    query = f"""
            SELECT * 
            FROM candles
            --WHERE timestamp >= NOW() - INTERVAL {hours} HOUR
            ORDER BY timestamp ASC
            """
    df = con.execute(query).fetchdf()
    con.close()
    return df


def compute_indicators(df):
    df.set_index('timestamp', inplace=True)

    # Add 3 basic indicators
    df['rsi'] = ta.rsi(df['close'], length=14)
    df['ema_50'] = ta.ema(df['close'], length=50)
    df['ema_200'] = ta.ema(df['close'], length=200)

    return df

def generate_signal(df):
    latest = df.iloc[-1]

    signal = "hold"
    reason = ""

    # Strategy logic
    if latest['ema_50'] > latest['ema_200'] and latest['rsi'] < 30:
        signal = "buy"
        reason = "Golden cross + RSI oversold"
    elif latest['ema_50'] < latest['ema_200'] and latest['rsi'] > 70:
        signal = "sell"
        reason = "Death cross + RSI overbought"

    return signal, reason

def get_ta_signal():
    df = load_recent_candles()
    df = compute_indicators(df)
    signal, reason = generate_signal(df)
    print(f"TA Signal: {signal.upper()} | Reason: {reason}")
    return signal, reason


In [ ]:
load_recent_candles()

In [ ]:
# Define the checkmark symbol using Unicode
checkmark = "\u2705"

# Display it to simulate a live conversion/output in a Python environment
checkmark
